# How to use chat models to call tools

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [Chat models](/docs/concepts/#chat-models)
- [LangChain Tools](/docs/concepts/#tools)
- [Output parsers](/docs/concepts/#output-parsers)

:::

:::info Tool calling vs function calling

We use the term tool calling interchangeably with function calling. Although
function calling is sometimes meant to refer to invocations of a single function,
we treat all models as though they can return multiple tool or function calls in 
each message.

:::

:::info Supported models

You can find a [list of all models that support tool calling](/docs/integrations/chat/).

:::

Tool calling allows a chat model to respond to a given prompt by "calling a tool".
While the name implies that the model is performing 
some action, this is actually not the case! The model generates the 
arguments to a tool, and actually running the tool (or not) is up to the user.
For example, if you want to [extract output matching some schema](/docs/how_to/structured_output/) 
from unstructured text, you could give the model an "extraction" tool that takes 
parameters matching the desired schema, then treat the generated output as your final 
result.

:::note

If you only need formatted values, try the [.with_structured_output()](/docs/how_to/structured_output/#the-with_structured_output-method) chat model method as a simpler entrypoint.

:::

However, tool calling goes beyond [structured output](/docs/how_to/structured_output/)
since you can pass responses from called tools back to the model to create longer interactions.
For instance, given a search engine tool, an LLM might handle a 
query by first issuing a call to the search engine with arguments. The system calling the LLM can 
receive the tool call, execute it, and return the output to the LLM to inform its 
response. LangChain includes a suite of [built-in tools](/docs/integrations/tools/) 
and supports several methods for defining your own [custom tools](/docs/how_to/custom_tools). 

Tool calling is not universal, but many popular LLM providers, including [Anthropic](https://www.anthropic.com/), 
[Cohere](https://cohere.com/), [Google](https://cloud.google.com/vertex-ai), 
[Mistral](https://mistral.ai/), [OpenAI](https://openai.com/), and others, 
support variants of a tool calling feature.

LangChain implements standard interfaces for defining tools, passing them to LLMs, 
and representing tool calls. This guide and the other How-to pages in the Tool section will show you how to use tools with LangChain.

## Passing tools to chat models

Chat models that support tool calling features implement a `.bind_tools` method, which 
receives a list of LangChain [tool objects](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool) 
and binds them to the chat model in its expected format. Subsequent invocations of the 
chat model will include tool schemas in its calls to the LLM.

For example, we can define the schema for custom tools using the `@tool` decorator 
on Python functions:

In [1]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

Or below, we define the schema using [Pydantic](https://docs.pydantic.dev):

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

We can bind them to chat models as follows:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  fireworksParams={`model="accounts/fireworks/models/firefunction-v1", temperature=0`}
/>
```

We'll use the `.bind_tools()` method to handle converting
`Multiply` to the proper format for the model, then and bind it (i.e.,
passing it in each time the model is invoked).

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain_openai

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [4]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g4RuAijtDcSeM96jXyCuiLSN', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 95, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5157d15a-7e0e-4ab1-af48-3d98010cd152-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_g4RuAijtDcSeM96jXyCuiLSN'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113})

As we can see, even though the prompt didn't really suggest a tool call, our LLM made one since it was forced to do so. You can look at the docs for [bind_tools()](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.BaseChatOpenAI.html#langchain_openai.chat_models.base.BaseChatOpenAI.bind_tools) to learn about all the ways to customize how your LLM selects tools.

## Tool calls

If tool calls are included in a LLM response, they are attached to the corresponding 
[message](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage) 
or [message chunk](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk) 
as a list of [tool call](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.ToolCall.html#langchain_core.messages.tool.ToolCall) 
objects in the `.tool_calls` attribute.

Note that chat models can call multiple tools at once.

A `ToolCall` is a typed dict that includes a 
tool name, dict of argument values, and (optionally) an identifier. Messages with no 
tool calls default to an empty list for this attribute.

In [5]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_TnadLbWJu9HwDULRb51RNSMw'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_Q9vt1up05sOQScXvUYWzSpCg'}]

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion, 
model providers may output malformed tool calls (e.g., arguments that are not 
valid JSON). When parsing fails in these cases, instances 
of [InvalidToolCall](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.InvalidToolCall.html#langchain_core.messages.tool.InvalidToolCall) 
are populated in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have 
a name, string arguments, identifier, and error message.

If desired, [output parsers](/docs/how_to#output-parsers) can further 
process the output. For example, we can convert existing values populated on the `.tool_calls` attribute back to the original Pydantic class using the
[PydanticToolsParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.openai_tools.PydanticToolsParser.html):

In [6]:
from langchain_core.output_parsers import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

[Multiply(a=3, b=12), Add(a=11, b=49)]

## Next steps

Now you've learned how to bind tool schemas to a chat model and to call those tools. Next, you can learn more about how to use tools:

- Few shot promting [with tools](/docs/how_to/tools_few_shot/)
- Stream [tool calls](/docs/how_to/tool_streaming/)
- Bind [model-specific tools](/docs/how_to/tools_model_specific/)
- Pass [runtime values to tools](/docs/how_to/tool_runtime)
- Pass [tool results back to model](/docs/how_to/tool_results_pass_to_model)

You can also check out some more specific uses of tool calling:

- Building [tool-using chains and agents](/docs/how_to#tools)
- Getting [structured outputs](/docs/how_to/structured_output/) from models